In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 26 19:34:58 2019

@author: Partha.Dip
"""

# -*- coding: utf-8 -*-
"""
Created on Wed Nov 20 16:07:44 2019

@author: Partha.Dip
"""

# -*- coding: utf-8 -*-
"""
Created on Tue Oct 22 11:01:33 2019

@author: Partha.Dip
"""
model=1

if model==1:
    modelConfiguration = "vocperson.cfg";
    modelWeights = "vocperson_132000.weights";     
    
elif model==2 :
   modelConfiguration = "tiny-yolov2-trial3-noBatch.cfg";
   modelWeights = "tiny-yolov2-trial3-noBatch.weights";

else:
    modelConfiguration = "yolov3-tiny.cfg";
    modelWeights = "yolov3-tiny.weights";


videopath="walk2.mp4"
#videopath="c.mp4"
videopath="webcam"



from time import time
import cv2 as cv
import sys
import numpy as np
import os.path


# Initialize the parameters
confThreshold = 0.9#Confidence threshold
#nmsThreshold = 0.4   #Non-maximum suppression threshold
nmsThreshold =0.5
#inpWidth = 416      #Width of network's input image
#inpHeight = 416      #Height of network's input image

#good
inpWidth = 224
inpHeight =128

#inpWidth = 392
#inpHeight =224

#inpWidth = 168
#inpHeight =98

#inpWidth = 224
#inpHeight =224
######extract frames##########
#########################

#videopath="walk2.mp4"



#videopath="c.mp4"
#videopath="webcam"
#videopath="model.mp4"
#videopath="www.mp4"


if videopath=="webcam":
    inpWidth = 224
    inpHeight= 224
#videopath="webcam"
#URL = "http://192.168.43.1:8080"

#tot_frame=0
#im = [0]*tot_frame



print(cv.__version__)
vidcap = cv.VideoCapture(videopath)
success,image = vidcap.read()
#count = 0
success = True

##################################
    
# Load names of classes
#classesFile = "coco.names";
classesFile = "inria.names";
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')




# load our serialized model from disk
print("[INFO] loading model...")
net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_OPENCL)

# initialize the frame dimensions (we'll set them as soon as we read
# the first frame from the video)
W = None
H = None
 



# Get the names of the output layers
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Draw the predicted bounding box
def drawPred(classId, conf, left, top, right, bottom):
    # Draw a bounding box.
    cv.rectangle(frame, (left, top), (right, bottom), (225, 0, 255), 2)
    

# Remove the bounding boxes with low confidence using non-maxima suppression
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    rects = []

    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    indices = cv.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        # Class "person"
        #if classIds[i] == 1 :#or classIds[i] == 0:
        rects.append((left, top, left + width, top + height))
            
        drawPred(classIds[i], confidences[i], left, top, left + width, top + height)
 
#def counting(objects):
    #frameHeight = frame.shape[0]
    #frameWidth = frame.shape[1]


# Process inputs
winName = 'Deep learning object detection in OpenCV'
cv.namedWindow(winName, cv.WINDOW_NORMAL)

#outputFile = "yolo_out_py.avi"

if (videopath!="webcam"):
    # Open the video file
    if not os.path.isfile(videopath):
        print("Input video file ", videopath, " doesn't exist")
        sys.exit(1)
    cap = cv.VideoCapture(videopath)
    
    
    
    
    
    #outputFile = videopath[:-4]+'_yolo_out_py.avi'
else:
    # Webcam input
    cap = cv.VideoCapture(0)

i=0
start_time = time()
while cv.waitKey(1) < 0:
    # get frame from the video
    hasFrame, frame = cap.read()
    if hasFrame==False:
        print("Done processing !!!")
        cv.destroyWindow(winName)
        cap.release()
        break 
    
    
    
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
    
    
    # Stop the program if reached end of video
       

    # Create a 4D blob from a frame.
    blob = cv.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)

    # Sets the input to the network
    net.setInput(blob)

    # Runs the forward pass to get output of the output layers
    outs = net.forward(getOutputsNames(net))

    # Remove the bounding boxes with low confidence
    postprocess(frame, outs)

    # Put efficiency information. The function getPerfProfile returns the overall time for inference(t) and the timings for each of the layers(in layersTimes)
    t, _ = net.getPerfProfile()
    label = 'Inference time: %.2f ms' % (t * 1000.0 / cv.getTickFrequency())
    
    cv.putText(frame, label, (0, 15), cv.FONT_HERSHEY_SIMPLEX, 0.5, (225, 0, 255))

    
    cv.imshow(winName, frame) 
    i+=1;
    #cv.waitKey(100)
    
end_time = time()  
seconds_elapsed = end_time - start_time

print(i/seconds_elapsed)
cv.destroyWindow(winName)
cap.release() 

4.1.1
[INFO] loading model...
8.374608601040604
